# iSEPTA Philly Project
## "The Tappiest Time of the Year" Contest

### by John A. Fonte

__Goal:__ Guess the total number of SEPTA key taps done on Friday, December 20, 2019 _without going over._

- __Definition:__ A SEPTA "key tap" is a payment for public transportation fare by way of "tapping" the recently introduced magnetic cards onto digital turnstiles. <br><br>
  - For purposes of this project, a "key tap" shall refer to a single use of a SEPTA key card for payment of fare.

__Link to Contest:__ https://www.iseptaphilly.com/contests/172

   - (_Update:_ The contest closed on December 20, 2019.)

***

## General Idea

We need to perform the following for our analysis:

1. Figure out what data to obtain.
2. Within the desired data, figure out what metric to use to determine SEPTA key taps.
3. Where to go to obtain said data.
4. Perform data analysis and data modeling, assuming that there is enough available relevant data.
5. Draw Conclusions

### 1. What Data includes SEPTA Key Card Taps?

SEPTA key taps are done for fare on all four modes of Philadelphia public transportation: 

   1. Subway (Market-Frankford Line (MFL) and Broad Street Line (BSL)) 
   2. Buses
   3. Trolleys 
   4. Regional Rail Lines
   
#### Data Limitation #1: No Historic Key Card Information for Regional Rail Lines

The use of SEPTA key cards is a recent change to Philadelphia's public transportation system, and indeed, SEPTA is still in the process of transitioning. Notably, the transition from standard payment to key taps for regional rail lines is [still in flux](https://www.inquirer.com/transportation/septa-key-regional-rail-rollout-travel-wallet-20191118.html). As such, there is not sufficient information to determine what portion of regional rail riders are currently using the SEPTA key card as the transition is so new (and quite frankly, it is unclear if this is calculated into the total number on iSEPTA Philly's end).

For this reason, we will not include regional rail ridership into the analysis.

#### Data Limitation #2: Fares Paid without SEPTA Key Card

Although the SEPTA key card has made fare tokens obsolete, individuals may still pay for fare via single "quick trip" purchases at digital kiosks or with cash via a SEPTA representative in person. As we will see, the data does not show a distinction between key tap ridership and non-key tap ridership

***


Don't use [this](https://www.dropbox.com/sh/3jnvonaqtmvc3wh/AACvwz3DMTXrW56P8xBUUIcSa?dl=0&preview=README.md) data (based on my exhaustive research, the json formatting is incorrect anyway). The real reason is that it has stats on the _train_ rides, nothing about rider*ship*.

In [14]:
import pandas as pd

In [18]:
df = pd.read_csv('D:/Github/Data-Science-Bootcamp/SEPTA-Competition-Project/SEPTA_Market_Frankford_Line.csv')

In [19]:
df.head()

,OBJECTID,Route,Vehicle_Hours,Vehicle_Miles,Peak_Vehicles,Average_Weekday_Passengers,Annual_Passengers,Average_Trip_Length,Passenger_Miles,Passenger_Revenue,Variable_Expenses,Variable_Recovery,Fully_Allocated_Expenses,Operating_Ratio,GlobalID,Shape__Length
0,1,Market-Frankford Line,470930,9230221,144,180512,54767414,4.7,257406800,59810392,137752084,158,136694484,44,9b7ec014-18d2-42af-a482-a86be8a370e6,0.237995
